### Consensus

In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "tseConsUp.RData"))
suppressPackageStartupMessages(source("tree_helper_function.R"))

In [2]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRVUp.RData"))
load(file.path(saveDir, "bouthBrain.RData"))
load(file.path(saveDir, "bSwishCons.RData"))

In [3]:
which(bouthBrain[[1]][["results.by.node"]][["is.driver"]])
which(bouthBrain[[2]][["results.by.node"]][["is.driver"]])
which(bouthBrain[[3]][["results.by.node"]][["is.driver"]])

[1] 1

[1] 1

[1] 1

In [23]:
load("environment/brain_sim_nodtu/mode=gc_bias/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tseConsUp)
l <- length(tree$tip)
missingTxps <- setdiff(tree$tip.label, rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[tree$tip.label,]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])

In [26]:
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
se <- tximeta::tximeta(coldata)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2023-03-02 17:08:07

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

loading existing transcript ranges created: 2023-03-02 21:45:10



In [13]:
sum(rownames(tseConsUp)[1:l] != tree$tip.label)

[1] 0

In [29]:
y <- se[tree$tip.label,]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))
detNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(tAfterBHMoreClimbMIRVUp[["mIRV=0.4"]], function(ta) ta)
detNodes[["treeClimbR"]] <- lapply(bSwish, function(sw) sw$output[sw$output$signal.node,][["node"]])
detNodes[["treeClimbRO"]] <- lapply(bSwish, function(sw) unlist(Descendants(tree,sw$output[sw$output$signal.node,][["node"]])))

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), nodes))
negNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes))))
negNodes[["treeClimbR"]] <- lapply(detNodes[["treeClimbR"]], function(det) setdiff(seq(nrow(y)), unlist(Descendants(tree, det))))
negNodes[["treeClimbRO"]] <- lapply(detNodes[["treeClimbRO"]], function(det) setdiff(seq(nrow(y)), det))

In [14]:
sw <- bSwish[[1]]
tail(sw$output[sw$output$signal.node,][["node"]])
tail(detNodes)

[1] 253823 253978 254002 254113 254122 254351

In [30]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{    
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
print(df)

[1] "Txps"
[1] "tp 0.990276157137301"
[1] "tp 0.962688916876574"
[1] "tp 0.92637778712663"
[1] "tAfterBHMoreClimbMIRV=0.4"
[1] "tp 0.997774952320407"
[1] "tp 0.969461697722567"
[1] "tp 0.93215966003946"
[1] "treeClimbR"
[1] "tp 0.997648902821317"
[1] "tp 0.984057174271578"
[1] "tp 0.957791345898593"
[1] "treeClimbRO"
[1] "tp 0.84087332808812"
[1] "tp 0.799852107468573"
[1] "tp 0.745443311708372"
                        Method Metric    FDR_0.01   FDR_0.05   FDR_0.10
fdr                       Txps    FDR 0.009723843 0.03731108 0.07362221
tpr                       Txps    TPR   0.5787679  0.6950443  0.7508525
fdr1 tAfterBHMoreClimbMIRV=0.4    FDR 0.002225048  0.0305383 0.06784034
tpr1 tAfterBHMoreClimbMIRV=0.4    TPR   0.4234453  0.7286993  0.7871332
fdr2                treeClimbR    FDR 0.002351097 0.01594283 0.04220865
tpr2                treeClimbR    TPR   0.3601641  0.6080163  0.6982775
fdr3               treeClimbRO    FDR   0.1591267  0.2001479  0.2545567
tpr3               treeCl

In [32]:
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]

In [33]:
dfA

Method,FDR_0.01_FDR,FDR_0.05_FDR,FDR_0.10_FDR,FDR_0.01_TPR,FDR_0.05_TPR,FDR_0.10_TPR
<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
Txps,0.009723843,0.03731108,0.07362221,0.5787679,0.6950443,0.7508525
tAfterBHMoreClimbMIRV=0.4,0.002225048,0.0305383,0.06784034,0.4234453,0.7286993,0.7871332
treeClimbR,0.002351097,0.01594283,0.04220865,0.3601641,0.6080163,0.6982775
treeClimbRO,0.1591267,0.2001479,0.2545567,0.4859059,0.7376677,0.8228006


In [34]:
detInnerNodes <- sapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) nodes[nodes > l])
sapply(detInnerNodes, length)
sapply(seq_along(detInnerNodes), function(i) mean(abs(logFCNodes[detInnerNodes[[i]]]) < rootFC))

[1] 1030 1227 1293

[1] 0.00000000 0.04563977 0.09899459

In [13]:
missP <- setdiff(unlist(Descendants(tree,detNodes[[3]][[2]])), detNodes[[1]][[2]])
missTP <- missP[abs(logFCNodes[missP]) > rootFC] ##True positive transcripts that have an internal node

missInTreeDE <- setdiff(detNodes[[1]][[2]], unlist(Descendants(tree,detNodes[[3]][[2]])))
missTPInTreeDE <- missInTreeDE[abs(logFCNodes[missInTreeDE]) > rootFC] ##True positive transcripts that have an internal node
sum(unique(unlist(Ancestors(tree, missTP))) %in% detNodes[[3]][[2]]) ### Nodes of these additional ancestors 

[1] 555

In [14]:
missP <- setdiff(unlist(Descendants(tree,detNodes[[3]][[2]])), detNodes[[1]][[2]])

In [15]:
length(missP)
length(missTP)
length(missInTreeDE)
length(missTPInTreeDE)

[1] 2219

[1] 790

[1] 284

[1] 204

In [16]:
dd <- sapply(Descendants(tree, detInnerNodes[[2]]), function(nodes) {
    if(length(nodes) > 1) {
        all(abs(logFCNodes)[nodes] > rootFC) & (sum(nodes %in% detNodes[[1]][[2]])==0)
    }
})

In [26]:
mcols(y)[Descendants(tree, detInnerNodes[[2]])[[146]],]

DataFrame with 2 rows and 8 columns
                  meanInfRV log10mean      keep      stat    log2FC     pvalue
                  <numeric> <numeric> <logical> <numeric> <numeric>  <numeric>
ENST00000508457.1   3.14153   1.63351      TRUE    -13.90  -1.00361 0.00815617
ENST00000379205.4   3.06227   1.65498      TRUE    -13.01  -0.90171 0.01409578
                     locfdr    qvalue
                  <numeric> <numeric>
ENST00000508457.1  0.517371 0.0744448
ENST00000379205.4  0.686017 0.1185206

In [24]:
sum(dd)
which(dd)[1:10]
logFCNodes[Descendants(tree, detInnerNodes[[2]])[[146]]]
which(assays(tse))

[1] 25

[1] 146 189 229 246 250 293 315 364 388 450

ENST00000508457.1 ENST00000379205.4 
        -1.162596         -1.157432

In [31]:
load(file=file.path(saveDir, "tseMean.RData"))
load(file.path(saveDir, "tAfterBHMoreClimbMIRVMean.RData"))

In [43]:
load("environment/brain_sim_nodtu/mode=gc_bias/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])

In [44]:
rootFC

Node138765 
0.03264375

In [34]:
y <- tse[1:l,]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

In [35]:
tAft

class: TreeSummarizedExperiment 
dim: 258730 12 
metadata(8): tximetaInfo quantInfo ... txpsAnn infRepsScaled
assays(105): counts abundance ... mean variance
rownames(258730): ENST00000419160.3 ENST00000416931.1 ... Node258729
  Node258730
rowData names(1): meanInfRV
colnames(12): 1_1 2_1 ... 5_2 6_2
colData names(2): names condition
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):
rowLinks: a LinkDataFrame (258730 rows)
rowTree: 1 phylo tree(s) (138764 leaves)
colLinks: NULL
colTree: NULL

In [79]:
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))
detNodes[["tAfterBHMoreClimbMIRV=0"]] <- lapply(tAfterBHMoreClimbMIRV[[1]], function(ta) ta)
detNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(tAfterBHMoreClimbMIRV[[2]], function(ta) ta)
detNodes[["tAfterBHMoreClimbMIRV=1"]] <- lapply(tAfterBHMoreClimbMIRV[[3]], function(ta) ta)

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), nodes))
negNodes[["tAfterBHMoreClimbMIRV=0"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes))))
negNodes[["tAfterBHMoreClimbMIRV=0.4"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes)))) 
negNodes[["tAfterBHMoreClimbMIRV=1"]] <- lapply(detNodes[["tAfterBHMoreClimbMIRV=1"]], function(nodes) setdiff(seq(nrow(y)), unlist(Descendants(tree, nodes)))) 


In [80]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{    
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
dfA                     

[1] "Txps"
[1] "tp 0.990465071025491"
[1] "tp 0.962445003142678"
[1] "tp 0.926497404965633"
[1] "tAfterBHMoreClimbMIRV=0"
[1] "tp 0.985894798707023"
[1] "tp 0.943812061031727"
[1] "tp 0.892118863049096"
[1] "tAfterBHMoreClimbMIRV=0.4"
[1] "tp 0.990968283973955"
[1] "tp 0.964466150218929"
[1] "tp 0.930474040632054"
[1] "tAfterBHMoreClimbMIRV=1"
[1] "tp 0.991990141712877"
[1] "tp 0.967318114632945"
[1] "tp 0.935857918684867"


Method,FDR_0.01_FDR,FDR_0.05_FDR,FDR_0.10_FDR,FDR_0.01_TPR,FDR_0.05_TPR,FDR_0.10_TPR
<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
Txps,0.009534929,0.037555,0.0735026,0.5784748,0.6961018,0.7506535
tAfterBHMoreClimbMIRV=0,0.0141052,0.05618794,0.1078811,0.6718062,0.7786214,0.8239857
tAfterBHMoreClimbMIRV=0.4,0.009031716,0.03553385,0.06952596,0.6215255,0.7392539,0.7893527
tAfterBHMoreClimbMIRV=1,0.008009858,0.03268189,0.06414208,0.595561,0.71803,0.7720063


In [81]:
detInnerNodes <- sapply(detNodes[["tAfterBHMoreClimbMIRV=0.4"]], function(nodes) nodes[nodes > l])
sapply(detInnerNodes, length)
sapply(seq_along(detInnerNodes), function(i) mean(abs(logFCNodes[detInnerNodes[[i]]]) < rootFC))

[1] 1122 1200 1251

[1] 0.01158645 0.05916667 0.10471623

In [78]:
tAfterBHMoreClimbMIRV[[1]] <- lapply(tAfterBHMoreClimbMIRV[[1]], function(nodes) nodes[!is.na(nodes)])
tAfterBHMoreClimbMIRV[[2]] <- lapply(tAfterBHMoreClimbMIRV[[2]], function(nodes) nodes[!is.na(nodes)])
tAfterBHMoreClimbMIRV[[3]] <- lapply(tAfterBHMoreClimbMIRV[[3]], function(nodes) nodes[!is.na(nodes)])

In [82]:
missP <- setdiff(unlist(Descendants(tree,detNodes[[3]][[2]])), detNodes[[1]][[2]])
missTP <- missP[abs(logFCNodes[missP]) > rootFC] ##True positive transcripts that have an internal node

missInTreeDE <- setdiff(detNodes[[1]][[2]], unlist(Descendants(tree,detNodes[[3]][[2]])))
missTPInTreeDE <- missInTreeDE[abs(logFCNodes[missInTreeDE]) > rootFC] ##True positive transcripts that have an internal node

sum(unique(unlist(Ancestors(tree, missTP))) %in% detNodes[[3]][[2]]) ### Nodes of these additional ancestors 
length(missP)
length(missTP)
length(missInTreeDE)
length(missTPInTreeDE)

[1] 548

[1] 2204

[1] 789

[1] 194

[1] 135